# Predicting house prices using k-nearest neighbors regression
In this notebook, you will implement k-nearest neighbors regression. You will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

# Fire up GraphLab Create

In [57]:
import sframe as sf
#import graphlab as gl
import numpy as np # note this allows us to refer to numpy as np instead

# Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [58]:
sales = sf.SFrame('./kc_house_data_small.gl/')
sales.head(2)

Columns:
	id	str
	date	datetime
	price	int
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	float
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 2

Data:
+------------+---------------------------+--------+----------+-----------+
|     id     |            date           | price  | bedrooms | bathrooms |
+------------+---------------------------+--------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 | 538000 |   3.0    |    2.25   |
+------------+---------------------------+--------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |  1.0   |     0      |  0   |     3     |   7   |    1180    |
|    2570.0   |   7242   |  2.0   |     0      |  0   |     3     |   7   |    2170    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|       0       |   1955   |      0       |  98178  | 47.51123398 |
|      400      |   1951   |     1991     |  98125  | 47.72102274 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
|  -122.3188624 |     1690.0    | ... |
+---------------+---------------+-----+
[2 rows x 21 columns]

# Import useful functions from previous notebooks

To efficiently compute pairwise distances among data points, we will convert the SFrame into a 2D Numpy array. First import the numpy library and then copy and paste `get_numpy_data()` from the second notebook of Week 2.

In [59]:
def get_numpy_data(data_sframe, features, output):
    #   this is how we call this function
    #   (example_features, example_output) = 
    #   get_numpy_data(sales, ['sqft_living'], 'price')
    #   sales is the SFrame, features_list = ['sqft_living', ....], response = 'price')
    
    data_sframe['constant'] = 1 # this is how you add a new column to an SFrame, tis case 'constant'
    
    # add the column 'constant' to the front of the features list 
    # so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # features = 'constant' + features # this wont work!! - string constant add to a list
    
    # print "print features", features, "\n"   prints nothing!!!
    
    # select the columns of data_SFrame given by the features list 
    # into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # features_sframe = gl.SFrame.to_dataframe(data_sframe[features])  #if use gl.SFrame
    # features_sframe = sf.SFrame.to_dataframe(data_sframe[features])
    
    #print "get_numpy_data(): print data_sframe = features_sframe(head 5)\n",features_sframe.head(5), "\n"  
    #prints the 2 columns constant and sqft_living
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    #len(feature_matrix)
    #print "get_numpy_data(): print data_sframe converted to feature_matrix[0:5, :]\n",feature_matrix[0:5, :] , "\n" 
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    
    return(feature_matrix, output_array)

We will also need the `normalize_features()` function from Week 5 that normalizes all feature columns to unit norm. Paste this function below.

In [60]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/ norms
    
    return  (normalized_features, norms)

# Split data into training, test, and validation sets

In [61]:
(train_and_validation, test) = sales.random_split(.8, seed=1) # initial train/test split
(train, validation) = train_and_validation.random_split(.8, seed=1) # split training set into training and validation sets

# Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transform the training, test, and validation SFrames into Numpy arrays:

In [62]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [63]:
features_train, norms = normalize_features(features_train) # normalize training set features (columns)
features_test = features_test / norms # normalize test set by training set norms
features_valid = features_valid / norms # normalize validation set by training set norms

# Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1.

In [64]:
# features_test[:,0]    this is 1st column idiot
# print len(features_test[:,0]) gave 1741
features_test[0,:]    #this is 1st row   - NB the ,: can be dropped if we not splicing the columns
print len(features_test[0,:]) #gave 1741
print features_test[0] 

18
[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059    0.
  0.05102365  0.0116321   0.01564352  0.01362084  0.02481682  0.01350306
  0.          0.01345386 -0.01346927  0.01375926  0.0016225 ]


Now print the 10th row (index 9) of the training feature matrix. Again, you get an 18-dimensional vector with components between 0 and 1.

In [65]:
features_train[9,:]    #this is 1st row
print len(features_train[9,:]) #gave 1741
print features_train[9] 

18
[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346254  0.01195898  0.00156612]


***QUIZ QUESTION ***

What is the Euclidean distance between the query house and the 10th house of the training set? 

Note: Do not use the `np.linalg.norm` function; use `np.sqrt`, `np.sum`, and the power operator (`**`) instead. The latter approach is more easily adapted to computing multiple distances at once.

In [79]:
print np.sqrt(np.sum( (features_test[0,:] - features_train[9,:])**2 ))

print np.sqrt(np.sum( (features_train[9] - features_test[0])**2 ))

0.0597235937167
0.0597235937167


# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

In [82]:
dist = list(xrange(10))
for i in xrange(10):
    dist[i] = np.sqrt(np.sum( (features_train[i] - features_test[0])**2 ))
    print dist[i]

0.060274709173
0.0854688114883
0.0614994643712
0.0534027397882
0.0584448406394
0.0598792151018
0.0546314049726
0.0554310832416
0.052383627841
0.0597235937167


*** QUIZ QUESTION ***

Among the first 10 training houses, which house is the closest to the query house?

In [83]:
np.min(dist)    #the 9th house

0.052383627840972731

It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [127]:
for i in xrange(3):
    print features_train[i]-features_test[0]
    dist[i] = np.sqrt(np.sum( (features_train[i] - features_test[0])**2 ))
    print dist[i]
    # should print 3 vectors (one for each training house) of length 18
    # there are 18 features for a row, so 18 element wise differences, 
    # one for each feature between test and each train row
    # THis thus implies that the distance computed is sum of absolute differences over all features
    # so we don't have feature wise differences!!

[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
   2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
  -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
  -5.14364795e-03   6.69281453e-04]
0.060274709173
[  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
   7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
  -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
  -2.89330197e-03   1.47606982e-03]
0.0854688114883
[  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
   1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
   0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
  -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
   3.72914476e-03   1.64764925e-03]
0.0614994643712


The subtraction operator (`-`) in Numpy is vectorized as follows: Here we compute the element-wise difference between the features of the query house (features_test[0]) and the first 3 training houses (features_train[0:3]) in one line, and just by giving a row splice [0:3] of train set!! too easy man...

In [111]:
print features_train[0:3] - features_test[0]

[[  0.00000000e+00  -3.87821276e-03  -1.20498190e-02  -1.05552733e-02
    2.08673616e-04  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03  -5.50336860e-03  -2.48168183e-02
   -1.63756198e-04   0.00000000e+00  -1.70072004e-05   1.30577772e-05
   -5.14364795e-03   6.69281453e-04]
 [  0.00000000e+00  -3.87821276e-03  -4.51868214e-03  -2.26610387e-03
    7.19763456e-04   0.00000000e+00   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -3.47633726e-03   1.30705004e-03  -1.45830788e-02
   -1.91048898e-04   6.65082271e-02   4.23240653e-05   6.22415897e-06
   -2.89330197e-03   1.47606982e-03]
 [  0.00000000e+00  -7.75642553e-03  -1.20498190e-02  -1.30002801e-02
    1.60518166e-03  -8.52950206e-03   0.00000000e+00  -5.10236549e-02
    0.00000000e+00  -5.21450589e-03  -8.32384500e-03  -2.48168183e-02
   -3.13866046e-04   0.00000000e+00   4.71047219e-05   1.56530415e-05
    3.72914476e-03   1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [128]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print results[0] - (features_train[0]-features_test[0])
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print results[1] - (features_train[1]-features_test[0])
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print results[2] - (features_train[2]-features_test[0])
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


Aside: it is a good idea to write tests like this cell whenever you are vectorizing a complicated operation.

# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house. So we just calculate element wise differences between test and all observations in training.

In [162]:
diff = features_train[0:len(features_train)] - features_test[0]
diff[len(features_train) - 1]   #18 feature by feature difference between test and last train obs

array([  0.00000000e+00,  -3.87821276e-03,  -3.01245476e-03,
        -8.46807236e-03,  -1.24208957e-03,   8.52950206e-03,
         0.00000000e+00,  -5.10236549e-02,   0.00000000e+00,
        -1.73816863e-03,  -3.09564484e-03,  -2.48168183e-02,
         2.04695248e-04,   0.00000000e+00,   3.61763127e-05,
         3.22568982e-06,  -3.92203156e-03,  -1.01041218e-03])

To test the code above, run the following cell, which should output a value -0.0934339605842:

In [156]:
print len(features_train)
print diff[-1].sum() # sum of the feature differences between the query and last training house
print np.sum(diff[-1])
# both above should print -0.0934339605842

5527
-0.0934339605842
-0.0934339605842


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [163]:
print np.sum(diff**2, axis=1)[15] # take sum of squares across each row, and print the 16th sum
print np.sum(diff[15]**2) # print the sum of squares for the 16th row -- should be same as above

print (diff[15]**2).sum() == np.sum(diff[15]**2)

0.00330705902879
0.00330705902879
True


With this result in mind, write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Assign the result to a variable `distances`.

**Hint**: Do not forget to take the square root of the sum of squares.

In [164]:
distances = np.sqrt(np.sum(diff**2, axis=1)[0:len(features_train)])

To test the code above, run the following cell, which should output a value 0.0237082324496:

In [165]:
print distances[100] # Euclidean distance between the query house and the 101th training house
# should print 0.0237082324496

0.0237082324496


Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [167]:
def compute_distances(features_instances, features_query):
    size = len(features_instances)
    diff = features_instances[0:size] - features_query
    distances = np.sqrt(  np.sum(diff**2, axis=1)[0:size] )
    return distances

*** QUIZ QUESTIONS ***

1.  Take the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?   382
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [183]:
distances = compute_distances(features_train, features_test[2])
print min(distances)                  # whats the shortest Euclidean dist
print np.argmin(distances)            # index of the shortest Euclidean dist item in numpy array
print distances[np.argmin(distances)] # verify 

Value of the query house = $438000
0.00286049526751
382
0.00286049526751


In [200]:
print "Actual value of the query house = $%d" %test[2]['price']
print "Predicted value of the query house = $%d" %train[np.argmin(distances)]['price']
print "Error of $%d" %(test[2]['price'] - train[np.argmin(distances)]['price'])

Actual value of the query house = $438000
Predicted value of the query house = $249000
Error of $189000


# Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).   

'numpy.argsort(a, axis=-1, kind='quicksort', order=None)' 

Returns the indices that would sort an array, rather then give sorted values, it gives indices of sorted values... great!!!

In [191]:
def k_nearest_neighbors(k, features_instances, features_query):
    size = len(features_instances)
    diff = features_instances[0:size] - features_query
    distances = np.sqrt(  np.sum(diff**2, axis=1)[0:size] )
    indices_of_sorted_distances = np.argsort(distances)
    first_k_indices = indices_of_sorted_distances[0:k]
    return first_k_indices

*** QUIZ QUESTION ***

Take the query house to be third house of the test set (`features_test[2]`).  What are the indices of the 4 training houses closest to the query house?

In [192]:
knn_2 =  k_nearest_neighbors(4, features_train, features_test[2])
print knn_2

[ 382 1149 4087 3142]


## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**Hint**: You can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.

In [193]:
def k_nearest_neighbors_value(k, features_train, output_train, test_query):
    
    size = len(features_train)
    diff = features_train[0:size] - test_query
    distances = np.sqrt(  np.sum(diff**2, axis=1)[0:size] )
    indices_of_sorted_distances = np.argsort(distances)
    first_k_indices = list(indices_of_sorted_distances[0:k])
    pred_value = np.sum(output_train[first_k_indices])/k
    
    return pred_value

*** QUIZ QUESTION ***

Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

In [201]:
knn_value = k_nearest_neighbors_value(4, features_train, output_train, features_test[2])
print "Actual value of the query house = $%d" %test[2]['price']
print "Predicted value of the query house = $%d" %knn_value
print "Error of $%d" %(test[2]['price'] - knn_value)

Actual value of the query house = $438000
Predicted value of the query house = $413987
Error of $24013


Compare this predicted value using 4-nearest neighbors to the predicted value using 1-nearest neighbor computed earlier.

1-nn gave predicted value = \$249000, which was off by \$189,000. the 4_nn estimate is only out by \$24013

## Make multiple predictions

Write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function should return a set of predicted values, one for each house in the query set.

**Hint**: To get the number of houses in the query set, use the `.shape` field of the query features matrix.  ndarray.shape - gives Tuple of array dimensions.See [the documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [206]:
def k_nearest_neighbors_value(k, features_train, output_train, features_test):
    
    size = len(features_train)
    (len_test,col) = features_test.shape
    pred_value = list(xrange(len_test))
    
    for i in xrange(len_test):
        diff = features_train[0:size] - features_test[i]
        distances = np.sqrt(  np.sum(diff**2, axis=1)[0:size] )
        indices_of_sorted_distances = np.argsort(distances)
        first_k_indices = list(indices_of_sorted_distances[0:k])
        pred_value[i] = np.sum(output_train[first_k_indices])/k
    
    return pred_value

*** QUIZ QUESTION ***

Make predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`. 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

In [226]:
knn_val = k_nearest_neighbors_value(10, features_train, output_train, features_test[0:10])

#print min(features_test[0:10]['price'])                  
#print np.argmin(features_test[0:10]['price'])            
#print distances[np.argmin(distances)] # verify 
print "Actual prices of 10 houses from test set\n", test[0:10]['price']
print "\nKNN predicted prices for these 10 test houses using NN from train data\n",knn_val
print "\nIndex of the house in this query set that has the lowest predicted value", np.argmin(knn_val)

print "\nPredicted value for trhis house is $%d" %knn_val[np.argmin(knn_val)]

print sorted(knn_val)



Actual prices of 10 houses from test set
[650000, 485000, 438000, 535000, 785000, 975000, 287000, 355000, 305000, 518500]

KNN predicted prices for these 10 test houses using NN from train data
[881300, 431860, 460595, 430200, 766750, 667420, 350032, 512800, 484000, 457235]

Index of the house in this query set that has the lowest predicted value 6

Predicted value for trhis house is $350032
[350032, 430200, 431860, 457235, 460595, 484000, 512800, 667420, 766750, 881300]


## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

(Depending on your computing environment, this computation may take 10-15 minutes.)

To visualize the performance as a function of `k`, plot the RSS on the VALIDATION set for each considered `k` value:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

kvals = range(1, 16)
plt.plot(kvals, rss_all,'bo-')

***QUIZ QUESTION ***

What is the RSS on the TEST data using the value of k found above?  To be clear, sum over all houses in the TEST set.